# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


### 데이터 읽어오기


In [6]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


pd.set_option('display.max_columns', None)

Matplotlib is building the font cache; this may take a moment.


In [7]:
ROOT_DIR = "data"
RANDOM_STATE = 110

# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
train_data

FileNotFoundError: [Errno 2] No such file or directory: 'data\\train.csv'

전처리

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import pandas as pd

# 이미 train_data가 정의되고 로드되었다고 가정합니다.

# 'target' 열을 따로 저장
target = train_data['target'].copy()

# 'target' 열을 제외한 데이터프레임 생성
data_without_target = train_data.drop(columns=['target'])

# 범주형 변수를 숫자형으로 변환
for column in data_without_target.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data_without_target[column] = le.fit_transform(data_without_target[column])

# 수치형 변수와 범주형 변수 구분
numeric_features = data_without_target.select_dtypes(exclude=['object']).columns
categorical_features = data_without_target.select_dtypes(include=['object']).columns

# 수치형 데이터에서 결측치 처리
X_numeric = data_without_target[numeric_features]
# 모든 값이 결측치인 열을 0으로 채우기
all_missing_numeric_columns = X_numeric.columns[X_numeric.isnull().all()]
X_numeric[all_missing_numeric_columns] = X_numeric[all_missing_numeric_columns].fillna(0)

# 나머지 수치형 결측치 처리
imputer_numeric = SimpleImputer(strategy='mean')
X_numeric_imputed = pd.DataFrame(imputer_numeric.fit_transform(X_numeric), columns=X_numeric.columns)

# 범주형 데이터에서 결측치 처리
X_categorical = data_without_target[categorical_features]
# 모든 값이 결측치인 열을 -1로 채우기
all_missing_categorical_columns = X_categorical.columns[X_categorical.isnull().all()]
X_categorical[all_missing_categorical_columns] = X_categorical[all_missing_categorical_columns].fillna(-1)

# 나머지 범주형 결측치 처리
if not X_categorical.empty:
    imputer_categorical = SimpleImputer(strategy='most_frequent')
    X_categorical_imputed = pd.DataFrame(imputer_categorical.fit_transform(X_categorical), columns=X_categorical.columns)
else:
    X_categorical_imputed = X_categorical  # 범주형 데이터가 비어있지 않은 경우 원본 데이터 사용

# 결측치 처리된 데이터 결합
processed_data = pd.concat([X_numeric_imputed, X_categorical_imputed], axis=1)

# 원래의 'target' 열을 다시 결합
train_data = pd.concat([processed_data, target.reset_index(drop=True)], axis=1)

# 'target' 열의 값을 1.0은 'Normal', 0.0은 'AbNormal'로 변경
train_data['target'] = train_data['target'].replace({1.0: 'Normal', 0.0: 'AbNormal'})

# 'target' 열의 값 확인
print(train_data["target"].value_counts())


In [ ]:
'''from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import pandas as pd

# 이미 train_data가 정의되고 로드되었다고 가정합니다.

# 'target' 열을 따로 저장
target = train_data['target'].copy()

# 'target' 열을 제외한 데이터프레임 생성
data_without_target = train_data.drop(columns=['target'])

# 대부분 NaN인 열 제거 (90% 이상 NaN인 경우 제거)
threshold = 0.9
missing_ratio = data_without_target.isnull().mean()
columns_to_drop = missing_ratio[missing_ratio > threshold].index
data_without_target = data_without_target.drop(columns=columns_to_drop)

# 범주형 변수를 숫자형으로 변환
for column in data_without_target.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data_without_target[column] = le.fit_transform(data_without_target[column])

# 수치형 변수와 범주형 변수 구분
numeric_features = data_without_target.select_dtypes(exclude=['object']).columns
categorical_features = data_without_target.select_dtypes(include=['object']).columns

# 수치형 데이터에서 결측치 처리
X_numeric = data_without_target[numeric_features]
# 모든 값이 결측치인 열을 0으로 채우기
all_missing_numeric_columns = X_numeric.columns[X_numeric.isnull().all()]
X_numeric[all_missing_numeric_columns] = X_numeric[all_missing_numeric_columns].fillna(0)

# 나머지 수치형 결측치 처리
imputer_numeric = SimpleImputer(strategy='mean')
X_numeric_imputed = pd.DataFrame(imputer_numeric.fit_transform(X_numeric), columns=X_numeric.columns)

# 범주형 데이터에서 결측치 처리
X_categorical = data_without_target[categorical_features]
# 모든 값이 결측치인 열을 -1로 채우기
all_missing_categorical_columns = X_categorical.columns[X_categorical.isnull().all()]
X_categorical[all_missing_categorical_columns] = X_categorical[all_missing_categorical_columns].fillna(-1)

# 나머지 범주형 결측치 처리
if not X_categorical.empty:
    imputer_categorical = SimpleImputer(strategy='most_frequent')
    X_categorical_imputed = pd.DataFrame(imputer_categorical.fit_transform(X_categorical), columns=X_categorical.columns)
else:
    X_categorical_imputed = X_categorical  # 범주형 데이터가 비어있지 않은 경우 원본 데이터 사용

# 결측치 처리된 데이터 결합
processed_data = pd.concat([X_numeric_imputed, X_categorical_imputed], axis=1)

# 원래의 'target' 열을 다시 결합
train_data = pd.concat([processed_data, target.reset_index(drop=True)], axis=1)

# 'target' 열의 값을 1.0은 'Normal', 0.0은 'AbNormal'로 변경
train_data['target'] = train_data['target'].replace({1.0: 'Normal', 0.0: 'AbNormal'})

# 'target' 열의 값 확인
print(train_data["target"].value_counts())'''

In [ ]:
train_data

### 언더 샘플링


In [ ]:
normal_ratio = 1.0  # 1.0 means 1:1 ratio

df_normal = train_data[train_data["target"] == "Normal"]
df_abnormal = train_data[train_data["target"] == "AbNormal"]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

### 데이터 분할


In [ ]:
df_train, df_val = train_test_split(
    df_concat,
    test_size=0.3,
    stratify=df_concat["target"],
    random_state=RANDOM_STATE,
)


def print_stats(df: pd.DataFrame):
    num_normal = len(df[df["target"] == "Normal"])
    num_abnormal = len(df[df["target"] == "AbNormal"])

    print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}" + f" ratio: {num_abnormal/num_normal}")


# Print statistics
print(f"  \tAbnormal\tNormal")
print_stats(df_train)
print_stats(df_val)

## 3. 모델 학습


### 모델 정의


In [ ]:
model = RandomForestClassifier(n_estimators=42,random_state=2021)

### 모델 학습


In [ ]:
features = []

for col in df_train.columns:
    try:
        df_train[col] = df_train[col].astype(int)
        features.append(col)
    except:
        continue

train_x = df_train[features]
train_y = df_train["target"]


model.fit(train_x, train_y)

num_features = len(features)
print(f"Number of features: {num_features}")

In [ ]:
# Train set에 대한 성능
y_pred = model.predict(train_x)
acc = accuracy_score(y_true = train_y, y_pred =y_pred)
print("Train set에 대한 성능")  
print(f"정확도:{acc:0.4f}")  


val_x = df_val[features]
val_y = df_val["target"]

# Test set에 대한 성능
y_pred = model.predict(val_x)
acc = accuracy_score(y_true = val_y, y_pred =y_pred)
print("\n")  
print("Test set에 대한 성능")  
print(f"정확도:{acc:0.4f}")


In [ ]:
"""cv_scores = []
estimator_list =[i for i in range(0,50,1)]
for i in tqdm(range(0,50,1)):
    model = RandomForestClassifier(n_estimators=i+1,
                                 n_jobs=-1,
                                 random_state=2021)
    score = cross_val_score(model, train_x, train_y, cv=10, scoring = 'accuracy').mean()
    cv_scores.append(score)

best_e = [estimator_list[i] for i in range(len(cv_scores)) if cv_scores[i] == np.max(cv_scores)]
plt.figure(figsize=(20,10))
plt.legend(["Cross validation scores"], fontsize=20)
plt.plot(estimator_list,cv_scores, marker='o', linestyle='dashed')
plt.xlabel("the number of trees", fontsize=20 )
plt.ylabel("Accuracy", fontsize=20)
plt.title("Accuracy Score", fontsize=25)
plt.axvline(best_e[0], color='r', linestyle='--', linewidth=2)
plt.show()
print(f"최적의 tree개수:{(cv_scores.index(max(cv_scores)))+1}")
print("\n")  
print("Train set에 대한 성능 (10 fold cross validation)")  
print(f"정확도:{max(cv_scores):0.4f}")"""

In [ ]:
'''import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=FutureWarning)

model = RandomForestClassifier(random_state=2021)
param_grid = {
    'n_estimators': [50, 60, 70],
    'max_depth': [20, 30, 40],
    'max_leaf_nodes': [50, 100, 200],
    'criterion':['gini', 'entropy'],
    'max_features':['sqrt', 'log2']
}

CV_model = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, verbose=1 ,n_jobs=-1)
CV_model.fit(train_x, train_y)
# 최적의 하이퍼 파라미터 셋
CV_model.best_params_'''

In [ ]:
best_model = RandomForestClassifier(n_estimators=70,max_depth=20,
    max_leaf_nodes=300,
    criterion='gini',
    max_features='log2',random_state=2021)
best_model.fit(train_x, train_y)

In [ ]:
# Train set에 대한 성능
y_pred = best_model.predict(train_x)
acc = accuracy_score(y_true = train_y, y_pred =y_pred)
print("Train set에 대한 성능")  
print(f"정확도:{acc:0.4f}")  


val_x = df_val[features]
val_y = df_val["target"]

# Test set에 대한 성능
y_pred = best_model.predict(val_x)
acc = accuracy_score(y_true = val_y, y_pred =y_pred)
print("\n")  
print("Test set에 대한 성능")  
print(f"정확도:{acc:0.4f}")

## 4. 제출하기


### 테스트 데이터 예측


테스트 데이터 불러오기


In [ ]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))
test_data

In [ ]:
# 'target' 열을 따로 저장
target = test_data['target'].copy()

# 'target' 열을 제외한 데이터프레임 생성
data_without_target = test_data.drop(columns=['target'])

# 범주형 변수를 숫자형으로 변환
for column in data_without_target.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data_without_target[column] = le.fit_transform(data_without_target[column])

# 수치형 변수와 범주형 변수 구분
numeric_features = data_without_target.select_dtypes(exclude=['object']).columns
categorical_features = data_without_target.select_dtypes(include=['object']).columns

# 수치형 데이터에서 결측치 처리
X_numeric = data_without_target[numeric_features]
# 모든 값이 결측치인 열을 0으로 채우기
all_missing_numeric_columns = X_numeric.columns[X_numeric.isnull().all()]
X_numeric[all_missing_numeric_columns] = X_numeric[all_missing_numeric_columns].fillna(0)

# 나머지 수치형 결측치 처리
imputer_numeric = SimpleImputer(strategy='mean')
X_numeric_imputed = pd.DataFrame(imputer_numeric.fit_transform(X_numeric), columns=X_numeric.columns)

# 범주형 데이터에서 결측치 처리
X_categorical = data_without_target[categorical_features]
# 모든 값이 결측치인 열을 -1로 채우기
all_missing_categorical_columns = X_categorical.columns[X_categorical.isnull().all()]
X_categorical[all_missing_categorical_columns] = X_categorical[all_missing_categorical_columns].fillna(-1)

# 나머지 범주형 결측치 처리
if not X_categorical.empty:
    imputer_categorical = SimpleImputer(strategy='most_frequent')
    X_categorical_imputed = pd.DataFrame(imputer_categorical.fit_transform(X_categorical), columns=X_categorical.columns)
else:
    X_categorical_imputed = X_categorical  # 범주형 데이터가 비어있지 않은 경우 원본 데이터 사용

# 결측치 처리된 데이터 결합
processed_data = pd.concat([X_numeric_imputed, X_categorical_imputed], axis=1)

# 원래의 'target' 열을 다시 결합
test_data = pd.concat([processed_data, target.reset_index(drop=True)], axis=1)

# 'target' 열의 값을 1.0은 'Normal', 0.0은 'AbNormal'로 변경
test_data['target'] = test_data['target'].replace({1.0: 'Normal', 0.0: 'AbNormal'})

# 'target' 열의 값 확인
print(test_data["target"].value_counts())

In [ ]:
df_test_x = test_data[features]

for col in df_test_x.columns:
    try:
        df_test_x.loc[:, col] = df_test_x[col].astype(int)
    except:
        continue

In [ ]:
test_pred = model.predict(df_test_x)
test_pred

### 제출 파일 작성


In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**


In [ ]:
print(f"{train_data.shape},{test_data.shape},{df_sub.shape}")